In [2]:
import secrets
import random

from Crypto.Cipher import DES3
from Crypto.Util.Padding import pad, unpad

from generacion_llaves_COMPLETADO import generate_3des_key, generate_iv

In [ ]:
def encrypt_3des_cbc(plaintext: bytes, key: bytes, iv: bytes) -> bytes:
    """
    Cifra datos usando Triple DES (3DES) en modo CBC con padding PKCS#7.

    Args:
        plaintext: Datos a cifrar en bytes.
        key:       Clave 3DES de 16 o 24 bytes.
        iv:        Vector de inicialización de 8 bytes.

    Returns:
        Texto cifrado (bytes), múltiplo del tamaño de bloque (8 bytes).

    Example:
        >>> key = generate_3des_key(2)
        >>> iv = generate_iv(8)
        >>> plaintext = b"Mensaje secreto para 3DES"
        >>> ciphertext = encrypt_3des_cbc(plaintext, key, iv)
        >>> len(ciphertext) % 8
        0  # Debe ser múltiplo de 8 (tamaño de bloque de DES)
    """
    if len(key) not in (16, 24):
        raise ValueError("La clave 3DES debe ser de 16 o 24 bytes")
    if len(iv) != 8:
        raise ValueError("El IV para 3DES debe ser de 8 bytes")

    # Aplicar padding PKCS#7 al bloque de 8 bytes (tamaño de bloque de DES/3DES)
    texto_con_padding = pad(plaintext, DES3.block_size)

    # Crear el objeto cifrador con modo CBC y el IV proporcionado
    cifrador_3des = DES3.new(key, DES3.MODE_CBC, iv=iv)

    return cifrador_3des.encrypt(texto_con_padding)


In [4]:
def decrypt_3des_cbc(ciphertext: bytes, key: bytes, iv: bytes) -> bytes:
    """
    Descifra datos usando Triple DES (3DES) en modo CBC y elimina el padding PKCS#7.

    Args:
        ciphertext: Datos cifrados en bytes.
        key:        Clave 3DES de 16 o 24 bytes.
        iv:         Vector de inicialización de 8 bytes.

    Returns:
        Texto plano original (bytes) sin padding.

    Example:
        >>> key = generate_3des_key(2)
        >>> iv = generate_iv(8)
        >>> plaintext = b"Mensaje secreto"
        >>> ciphertext = encrypt_3des_cbc(plaintext, key, iv)
        >>> descifrado = decrypt_3des_cbc(ciphertext, key, iv)
        >>> descifrado == plaintext
        True
    """
    if len(key) not in (16, 24):
        raise ValueError("La clave 3DES debe ser de 16 o 24 bytes")
    if len(iv) != 8:
        raise ValueError("El IV para 3DES debe ser de 8 bytes")

    # Crear el objeto descifrador con el mismo modo CBC y el mismo IV
    descifrador_3des = DES3.new(key, DES3.MODE_CBC, iv=iv)

    # Descifrar y eliminar el padding PKCS#7
    texto_descifrado_con_relleno = descifrador_3des.decrypt(ciphertext)
    return unpad(texto_descifrado_con_relleno, DES3.block_size)


In [9]:
# Seleccionar tamaño de clave dinámicamente usando random
opcion_clave = random.choice([2, 3])
mi_clave = generate_3des_key(opcion_clave)
longitud_clave = 16 if opcion_clave == 2 else 24
print(f"Opción de clave seleccionada (random): {opcion_clave} ({longitud_clave} bytes)")
print(f"Clave 3DES: {mi_clave.hex()}")

# Generar IV aleatorio de 8 bytes usando secrets directamente
mi_iv = secrets.token_bytes(8)
print(f"IV (8 bytes): {mi_iv.hex()}")

print("\n")


# Mensaje de prueba
mensaje_original = b"Mensaje secreto para 3DES CBC"
print(f"Texto original  : {mensaje_original}")
print(f"Longitud        : {len(mensaje_original)} bytes")

print("\n")

# Cifrando
mensaje_cifrado = encrypt_3des_cbc(mensaje_original, mi_clave, mi_iv)
print(f"Texto cifrado   : {mensaje_cifrado.hex()}")
print(f"Longitud cifrado: {len(mensaje_cifrado)} bytes (múltiplo de 8: {len(mensaje_cifrado) % 8 == 0})")

# Descifrado
mensaje_descifrado = decrypt_3des_cbc(mensaje_cifrado, mi_clave, mi_iv)
print(f"Texto descifrado: {mensaje_descifrado}")
print(f"¿Coincide?      : {mensaje_descifrado == mensaje_original}")

print("\n")
# Demo con clave de 24 bytes si la anterior fue de 16, y viceversa
opcion_alterna = 3 if opcion_clave == 2 else 2
clave_alterna = generate_3des_key(opcion_alterna)
iv_alterno = generate_iv(8)
longitud_clave_alterna = 16 if opcion_alterna == 2 else 24
print(f"Demo alternativa con clave de {longitud_clave_alterna} bytes (opción {opcion_alterna}):")
print(f"Clave 3DES: {clave_alterna.hex()}")
print(f"IV (8 bytes): {iv_alterno.hex()}")

segundo_mensaje = b"Hola Mundo desde 3DES"
cifrado_2 = encrypt_3des_cbc(segundo_mensaje, clave_alterna, iv_alterno)
descifrado_2 = decrypt_3des_cbc(cifrado_2, clave_alterna, iv_alterno)
print(f"Original  : {segundo_mensaje}")
print(f"Cifrado   : {cifrado_2.hex()}")
print(f"Descifrado: {descifrado_2}")
print(f"¿Coincide?: {descifrado_2 == segundo_mensaje}")


Opción de clave seleccionada (random): 2 (16 bytes)
Clave 3DES: 083c9d60d8195160ff606a75240d4cbd
IV (8 bytes): 0a098253ab83278d


Texto original  : b'Mensaje secreto para 3DES CBC'
Longitud        : 29 bytes


Texto cifrado   : fb1ac53d68f36fa7fbe1e01e246521e365b72fbee92464829e04d48fdc8c614b
Longitud cifrado: 32 bytes (múltiplo de 8: True)
Texto descifrado: b'Mensaje secreto para 3DES CBC'
¿Coincide?      : True


Demo alternativa con clave de 24 bytes (opción 3):
Clave 3DES: a606931703bcf8c2ecb905ae31ab5c8d7b11445cb852769c
IV (8 bytes): b995d88ebae72981
Original  : b'Hola Mundo desde 3DES'
Cifrado   : 9267e5ea5032f4e3fd7226bd755ee4a7d5f666fba32a5a49
Descifrado: b'Hola Mundo desde 3DES'
¿Coincide?: True
